# Multiple sequence alignment and tree building

* __Notebook version__: `v0.0.1`
* __Created by:__ [Dr. Hiren Ghosh](https://www.linkedin.com/in/hiren-ghosh-phd-6181853a), `Imperial BRC Genomics Facility`
* __Maintained by:__ `Imperial BRC Genomics Facility`
* __Docker image:__ `TODO`
* __Github repository:__ [imperial-genomics-facility/viral-genome-notebook-image](https://github.com/imperial-genomics-facility/viral-genome-notebook-image)
* __Created on:__ `2020-April-21 14:42`
* __Contact us:__ [Imperial BRC Genomics Facility](https://www.imperial.ac.uk/medicine/research-and-impact/facilities/genomics-facility/contact/)
* __License:__ [Apache License 2.0](https://github.com/imperial-genomics-facility/scanpy-notebook-image/blob/master/LICENSE)

## Data fetching

Download the corona virus genome sequences mentioned in the paper [Complete Genome Sequence of a 2019 Novel Coronavirus (SARS-CoV-2) Strain Isolated in Nepal](https://mra.asm.org/content/9/11/e00169-20)

__List of genomes__:

* [__NC_045512.2__](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2): Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
* [__MN988668.1__](https://www.ncbi.nlm.nih.gov/nuccore/MN988668.1): Severe acute respiratory syndrome coronavirus 2 isolate 2019-nCoV WHU01, complete genome
* [__MN938384.1__](https://www.ncbi.nlm.nih.gov/nuccore/MN938384.1): Severe acute respiratory syndrome coronavirus 2 isolate 2019-nCoV_HKU-SZ-002a_2020, complete genome
* [__MN975262.1__](https://www.ncbi.nlm.nih.gov/nuccore/MN975262.1): Severe acute respiratory syndrome coronavirus 2 isolate 2019-nCoV_HKU-SZ-005b_2020, complete genome
* [__MN985325.1__](https://www.ncbi.nlm.nih.gov/nuccore/MN985325.1): Severe acute respiratory syndrome coronavirus 2 isolate 2019-nCoV/USA-WA1/2020, complete genome
* [__MN988713.1__](https://www.ncbi.nlm.nih.gov/nuccore/MN988713.1): Severe acute respiratory syndrome coronavirus 2 isolate 2019-nCoV/USA-IL1/2020, complete genome
* [__MN994467.1__](https://www.ncbi.nlm.nih.gov/nuccore/MN994467.1): Severe acute respiratory syndrome coronavirus 2 isolate 2019-nCoV/USA-CA1/2020, complete genome
* [__MN994468.1__](https://www.ncbi.nlm.nih.gov/nuccore/MN994468.1): Severe acute respiratory syndrome coronavirus 2 isolate 2019-nCoV/USA-CA2/2020, complete genome
* [__MN997409.1__](https://www.ncbi.nlm.nih.gov/nuccore/MN997409.1): Severe acute respiratory syndrome coronavirus 2 isolate 2019-nCoV/USA-AZ1/2020, complete genome

In [1]:
import os
import requests
import json
import time
from ete3 import Tree, TreeStyle,SeqMotifFace
from PIL import Image
os.environ['QT_QPA_PLATFORM']='offscreen'

In [2]:
def fetch_genome_fasta_from_ncbi(refseq_id,output_path='.'):
  '''
  A function for fetching the genome fasta sequences from NCBI
  
  :param refseq_id: NCBI genome id
  :param output_path: Path to dump genome files, default '.'
  '''
  try:
    url = \
      'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id={0}&rettype=fasta'.\
        format(refseq_id)
    r = requests.get(url)
    fasta_data = r.content.decode('utf-8')
    output_file = \
      os.path.join(
        os.path.abspath(output_path),
        '{0}.fa'.format(refseq_id))
    with open(output_file,'w') as fp:
      fp.write(fasta_data)
    print('Downloaded genome seq for {0}'.format(refseq_id))
  except Exception as e:
    print('Failed to download data for {0} from NCBI, error: {1}'.format(refseq_id,e))

In [3]:
genome_list = [
    'MN988668.1',
    'NC_045512.2',
    'MN938384.1',
    'MN975262.1',
    'MN985325.1',
    'MN988713.1',
    'MN994467.1',
    'MN994468.1',
    'MN997409.1'
]

In [ ]:
for id in genome_list:
    fetch_genome_fasta_from_ncbi(id)
    time.sleep(2)

## Multiple sequence alignment with Muscle

Merge all fasta files to a single filne and create multiple sequence alignment

In [ ]:
%%bash
cat NC*.fa > corona.fa
cat MN*.fa >> corona.fa
grep '>' corona.fa|wc -l

In [ ]:
!muscle -in corona.fa -out muscle_out.afa

## Tree building using FastTree

Build phylogenetic tree with FastTree

In [ ]:
!FastTree -nt -gtr < muscle_out.afa >corona.tree

## Plotting trees

In [ ]:
t = Tree("corona.tree")

In [ ]:
## ascii tree
print(t)

In [ ]:
## plain tree
t.render("%%inline")

In [ ]:
## circular tree
ts = TreeStyle()
ts.mode = "c"
t.render("%%inline",  tree_style=ts)

In [ ]:
## tree with branch length
ts = TreeStyle()
ts.show_leaf_name = True
ts.show_branch_length = True
ts.show_branch_support = True
t.render("%%inline", tree_style=ts)

In [ ]:
## 180 deg circular tree
ts = TreeStyle()
ts.show_leaf_name = True
ts.mode = "c"
ts.arc_start = -180 # 0 degrees = 3 o'clock
ts.arc_span = 180
t.render("%%inline", tree_style=ts)

In [ ]:
## read aligned fastq file
fasta_data = dict()
with open('muscle_out.afa','r') as fp:
    header = ''
    fasta_list = list()
    for line in fp:
        line = line.strip()
        if line.startswith('>'):
            if header != '':
                fasta_data.update({header:''.join(fasta_list)})
            header = line.split()[0].replace('>','')
            fasta_list = list()
        else:
            fasta_list.append(line)
    
    fasta_data.update({header:''.join(fasta_list)})

In [ ]:
## tree with aligned seq
for seq_id,seq in fasta_data.items():
    seqFace = SeqMotifFace(seq, gapcolor="red")
    (t & "{0}".format(seq_id)).add_face(seqFace, 0, "aligned")
ts = TreeStyle()
ts.tree_width = 100
t.render("tree_with_aln.png", tree_style=ts);
## check png using image viewer